In [1]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from utils_cosm import MHI_Modi2019, MHI_Padmanabhan2017
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [13]:
i_run, i_z = 6, 1 # This means we work using LH_0 and z = 2.46

path_in = './outputs/LH_%d/' %(i_run)
path_out = path_in

redshift = np.loadtxt('./outputs/redshifts.txt', usecols=(1))
params = np.loadtxt('./outputs/params_IllustrisTNG.txt')

z = redshift[i_z]
#t_back = cosmo.lookback_time(z).to(u.Myr)
print('z=%.3f' %(z))

print('Om0	sigma8	Asn1	Aagn1	Asn2	Aagn2')
print(params[i_run])

z=2.460
Om0	sigma8	Asn1	Aagn1	Asn2	Aagn2
[0.2878  0.785   1.8226  0.77378 1.60993 0.98282]


In [3]:
# Importing data
f = h5py.File('%sMHI_LH%d_z=%.3f.hdf5' %(path_in, i_run, z))
mHalo = f['MassHalo'][:]
mHI = f['M_HI'][:]
print(mHalo.size)
f.close()

24671


In [14]:
# Defining feature names

new_features = ['MassHalo','VelHalo','Nsubs','MassBH','dotMassBH','PosHalo',
                'MetalsGas_0','MetalsGas_1','MetalsGas_2','MetalsStar_0',
                'MetalsStar_1','MetalsStar_2','SFR','M_HI']

# Defining a new dataset

f = h5py.File('%sMHI_LH%d_z=%.3f.hdf5' %(path_in, i_run, z))

MHalo = f['MassHalo']
VelHalo = f['VelHalo']
Nsubs = f['Nsubs']
MassBH = f['MassBH']
dotMassBH = f['dotMassBH']
posHalo = f['PosHalo']
MetalsGas = f['MetalsGas']
MetalsStar = f['MetalsStar']
SFR = f['SFR']
M_HI = f['M_HI']

data = np.empty((MHalo.size,len(new_features)))

data[:,0] = MHalo

# Instead of saving the velocity of the Halo among each canonical direction, we decide to summarize them by compute 
# the norm of the valocity. By doing so, we not only reduce the dimension of our data (extremely useful to avoid the 
# curse of dimensionality), but we ensure to reduce the computation without losing useful information.

data[:,1] = np.linalg.norm(VelHalo, axis = 1)
data[:,2] = Nsubs
data[:,3] = MassBH
data[:,4] = dotMassBH

# Since the model does not take into account the interaction among different Halos, there is no need to save the coordinates
# along different dimension but it is enough to save the norm of the vector indicating the distance of the Halo from the
# centre of the simulation

data[:,5] = np.linalg.norm(posHalo, axis = 1)

data[:,6] = MetalsGas[:,0]
data[:,7] = MetalsGas[:,1]
data[:,8] = MetalsGas[:,2]
data[:,9] = MetalsStar[:,0]
data[:,10] = MetalsStar[:,1]
data[:,11] = MetalsStar[:,2]
data[:,12] = SFR
data[:,13] = M_HI

df_original = pd.DataFrame(data, columns = new_features)
print(df_original.describe())
df_original.sample(5)
df = df_original.copy()

           MassHalo       VelHalo         Nsubs        MassBH     dotMassBH  \
count  2.379900e+04  23799.000000  23799.000000  2.379900e+04  2.379900e+04   
mean   9.923163e+09    160.378952      0.934241  7.105525e+04  8.362755e+04   
std    5.375802e+10     69.182700      0.798932  2.445248e+06  3.929155e+06   
min    4.112524e+08      5.241392      0.000000  0.000000e+00  0.000000e+00   
25%    1.828179e+09    111.716755      1.000000  0.000000e+00  0.000000e+00   
50%    2.927628e+09    153.349869      1.000000  0.000000e+00  0.000000e+00   
75%    6.140109e+09    200.546677      1.000000  0.000000e+00  0.000000e+00   
max    3.165422e+12    526.323792     31.000000  2.507143e+08  3.684733e+08   

            PosHalo   MetalsGas_0   MetalsGas_1   MetalsGas_2  MetalsStar_0  \
count  23799.000000  23799.000000  23799.000000  2.379900e+04  23799.000000   
mean      23.444899      0.759848      0.240043  6.624875e-05      0.071861   
std        6.910161      0.004947      0.001571  2.

#### PROBLEMS

We noticed that the following features have value zero in the majority of the observations  
- SFR
- MassBH
- dotMassBH (that is strictly correlated with the one above)

In [15]:
print(df['SFR'].value_counts())
print('\n the total number of observation is {}'.format(df.shape[0]))

0.000000    21479
0.158912        1
0.006673        1
0.015949        1
0.270804        1
            ...  
0.027989        1
0.057930        1
0.108045        1
0.027011        1
0.010493        1
Name: SFR, Length: 2321, dtype: int64

 the total number of observation is 23799


In [16]:
print(df['MassBH'].value_counts())
print('\n the total number of observation is {}'.format(df.shape[0]))

0.000000e+00    23031
8.000018e+05        2
8.000011e+05        2
8.000091e+05        2
8.001593e+05        2
                ...  
1.743808e+07        1
8.070761e+05        1
8.725911e+05        1
8.069633e+05        1
8.025422e+05        1
Name: MassBH, Length: 763, dtype: int64

 the total number of observation is 23799


In [17]:
print(df['dotMassBH'].value_counts())
print('\n the total number of observation is {}'.format(df.shape[0]))

0.000000         23031
134934.328125        1
1285.847778          1
2088.588867          1
35465.093750         1
                 ...  
160915.156250        1
12616.737305         1
17040.324219         1
5796.000977          1
1007.103455          1
Name: dotMassBH, Length: 769, dtype: int64

 the total number of observation is 23799
